In [4]:
import random
import numpy as np
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import pandas as pd
import sys
import io
import copy
from tqdm.notebook import tqdm  # tqdm.notebookをインポート
from IPython.display import display, HTML
import cProfile


sys.path.append("../")
from models import Cars, ReservationTable, DFRSimulation, VFRSimulation, SuddenStopDFRSimulation
from utils import check_multiple_noise_effect, validate_with_ttc, generate_arrival_times_poisson

# ↑↑↑↑↑↑↑↑↑この部分は別セルでも良い↑↑↑↑↑↑↑↑↑

def create_arrival_times_arithmetic_seq(step, n_max):
    return [i * step for i in range(n_max)]

def arithmetic_sequence(first_term, common_difference, num_terms):
    return [first_term + common_difference * n for n in range(num_terms)]

# 初期設定（独立変数群）
CAR_NUM = 300 #時間窓内の車の総数
START_TIME = 0
TOTAL_LENGTH = 3000 # 入り口から出口までの距離 [m]
NOISE_START_X = 1850
OBSERVATION_POINTS = [600, NOISE_START_X-1, 1950]

# 流量カウント用
flow_count_interval = 5
SEGMENT_LENGTH=100
flow_calc_start = 100 # 何秒後から流量の計測を始めるか


WAYPOINTS_NUM = 20 #constrained way pointの数
MEAN_SPEED = 20 # 秒速[m/s]として定義
MAX_SPEED = 30 # 秒速[m/s]として定義
MAX_ACC = 3 # m/s^sで定義, maxは0.3gにした
TIME_STEP = 0.1
PLOT_CONDITION=["noise_continue", "noise_created"]
# TOTAL_TIME = 100 # シミュレーション時間

# 研究用のパラメータ
TTC = 1.6 #車間時間を暫定的に定義
COMMUNICATION_SPEED = 0
LAMBDA = 3 # 台数密度を車間時間として定義

# ノイズ関連のパラメ-タ
NOISE_FREQUENCY = 19

arrival_times = create_arrival_times_arithmetic_seq(LAMBDA, CAR_NUM)
# arrival_times = [2.805611314755843, 4.8056113147558435, 12.45418334124825, 17.61211498044407, 21.674961724245826, 23.674961724245826, 25.674961724245826, 31.01738642384915, 33.0355321495912, 35.0355321495912, 37.0355321495912, 50.72232545890806, 53.28195113722663, 58.643046837549164, 60.643046837549164, 62.643046837549164, 66.3180561603613, 69.90570543546659, 71.90570543546659, 75.80826562583078, 83.18906347590797, 85.85451992305059, 87.85451992305059, 92.5756159681414, 94.5756159681414, 96.5756159681414, 98.5756159681414, 101.6651765089501, 103.6651765089501, 110.05413309381916, 113.78187314652725, 120.70411201001693, 124.5444753695085, 128.69066124619636, 130.69066124619636, 132.9643533178792, 136.10646274385812, 138.10646274385812, 140.10646274385812, 142.10646274385812, 144.21750870897847, 146.21750870897847, 149.8726371315692, 154.36629397943378, 157.73429300337168, 159.73429300337168, 166.10515448927555, 170.76587981468654, 177.6172249091577, 181.20674538328223, 183.20674538328223, 191.0856979310541, 195.7951292631883, 197.7951292631883, 199.7951292631883, 201.7951292631883, 203.7951292631883, 205.7951292631883, 210.84675914652456, 213.02016499169758, 216.13196411790912, 221.7631507383874, 223.7631507383874, 229.86053417345707, 231.86053417345707, 234.6564081083129, 240.25195126297137, 242.25195126297137, 244.7410730950167, 246.7410730950167, 250.32172232537462, 252.32172232537462, 254.79385093464091, 259.7887572694224, 261.7887572694224, 263.7887572694224, 265.7887572694224, 267.7887572694224, 269.7887572694224, 271.7887572694224, 277.411112520444, 281.9025054780484, 285.3708119307591, 288.9544430319777, 291.3127945778801, 293.3127945778801, 295.7890822041264, 299.79472825514387, 301.79472825514387, 303.79472825514387, 305.79472825514387, 307.8751945681391, 319.9547903024209, 322.1483555447243, 331.8288493420763, 348.5939101025975, 350.5939101025975, 352.5939101025975, 358.8615012107836, 360.8615012107836, 362.8615012107836, 365.69176010087796, 367.69176010087796, 380.8372263074677, 382.8372263074677, 385.691319886154, 394.9881156170315, 396.9881156170315, 400.68180490817275, 405.72720568461085, 409.7007037873151, 411.7007037873151, 415.1521106057683, 423.02322621117526, 428.3079852334132, 430.3079852334132, 434.31030792257644, 439.17473045212006, 441.30347417169185, 451.2960346943851, 459.8101482411158, 462.81830365416147, 464.81830365416147, 467.7967723288717, 469.7967723288717, 483.7104421950171, 485.7104421950171, 492.8950625540238, 494.8950625540238, 496.8950625540238, 498.92755939791897, 500.92755939791897, 508.5657624659219, 521.0310470290151, 523.1386398660222, 525.3410414471647, 554.3929044421307, 556.3929044421307, 558.3929044421307, 560.3929044421307, 562.3929044421307, 564.3929044421307, 567.9024564446015, 583.0467713239962, 585.0467713239962, 587.0467713239962, 592.27702203698, 596.8457966827882, 599.677970487058, 601.677970487058, 603.677970487058, 605.677970487058, 613.1543429045331, 619.879002414836, 624.5867848690966, 630.0694935928452, 632.0694935928452, 634.0694935928452, 639.7658686039352, 641.7658686039352, 666.6771781072678, 668.6771781072678, 670.6771781072678, 672.6771781072678, 682.2497900086923, 686.0672401944468, 692.6521960547575, 702.7151508374712, 705.7258074984599, 709.6871632249706, 713.2132603948002, 715.2132603948002, 720.1206752688595, 722.1206752688595, 729.8852133346647, 732.0306195109456, 734.0306195109456, 737.7301500421327, 739.7301500421327, 747.1990405287322, 749.8085855513195, 752.5008968312366, 756.9351794690243, 759.5399850821067, 762.8033006845096, 764.8033006845096, 770.182583910496, 772.182583910496, 774.3256284907151, 788.7507398509098, 790.7507398509098, 792.7507398509098, 795.4177302451491, 797.4177302451491, 800.3675501176599, 802.3675501176599, 804.3675501176599, 811.2721607726187, 813.2721607726187, 815.3417210412985]

car_params = {"v_mean":MEAN_SPEED, "v_max": MAX_SPEED, "a_max":MAX_ACC, "a_min":MAX_ACC}

noise_params ={"NOISE_FREQUENCY":NOISE_FREQUENCY, "NOISE_START_X":NOISE_START_X}
brake_params={ "decel": -2, "acc":2, "coast_time":2, "period": 3}

def register_initial_eta(cars):
    for idx, car in enumerate(CARS):
        desired_eta_list = car.create_desired_eta(WAYPOINTS)
        is_valid = reservation_table.validate(desired_eta_list)
        if is_valid:
            reservation_table.register(desired_eta_list)
        else:
            calibration_info = {"desired_list": desired_eta_list}
            calibrated_list, _ = reservation_table.calibrate_list(**calibration_info)
            # print(calibrated_list)
            # print()
            reservation_table.register(calibrated_list)
            car.my_list = calibrated_list

print(arrival_times[-1]-1)
TOTAL_TIME = min(arrival_times[-1]-1, 600)

896


In [5]:
# ここからDFRのシミュレーション
# 従属的に決まるもの
WAYPOINTS = [{"waypoint_idx": i, "x": TOTAL_LENGTH / WAYPOINTS_NUM * (i)} for i in range(WAYPOINTS_NUM+1)]
group_id = 0
CARS = [Cars(arrival_time=time, index=index, v_mean=MEAN_SPEED, v_max = MAX_SPEED, a_max=MAX_ACC, a_min=MAX_ACC)
              for index, time in enumerate(arrival_times)]

reservation_table = ReservationTable(waypoints = WAYPOINTS, global_params = {"DESIRED_TTC":TTC})

simulation = SuddenStopDFRSimulation(TOTAL_TIME = TOTAL_TIME, TIME_STEP = TIME_STEP, TOTAL_LENGTH = TOTAL_LENGTH, CARS = CARS, TTC=TTC, \
                          reservation_table = reservation_table, noise_params=noise_params, brake_params=brake_params, COMMUNICATION_SPEED=COMMUNICATION_SPEED, DENSITY=LAMBDA, \
                          observation_points = OBSERVATION_POINTS, segment_length=SEGMENT_LENGTH, flow_count_interval=flow_count_interval)

# simulation.conduct_simulation(should_plot=True, plot_start=100, plot_finish=200)


In [6]:
#simulation.conduct_simulation(should_plot=False, plot_start=560, plot_finish=580)
cProfile.run('simulation.conduct_simulation(should_plot=True, plot_start=0, plot_finish=130 )', 'profile_result.prof')
import pstats
stats = pstats.Stats('profile_result.prof')
stats.strip_dirs().sort_stats('cumulative').print_stats(20)  # 上位20件を表示

Simulation Progress DFR, density=3:   0%|          | 0/6000 [00:00<?, ?it/s]

t=19.00, sudden_brake発生, 対象車: 0
ID: 0, 元のacc_itinerary: [{'acc': 0, 't_start': 0, 'v_0': 20, 't_end': 10000000.0, 'x_start': 0}]
挿入後:  [{'acc': 0, 't_start': 0, 'v_0': 20, 't_end': 19.0, 'x_start': 0}, {'t_start': 19.0, 't_end': 22.0, 'acc': -2, 'v_0': 20.0, 'x_start': 380.0}, {'t_start': 22.0, 't_end': 24.0, 'acc': 0, 'v_0': 14.0, 'x_start': 431.0}, {'t_start': 24.0, 't_end': 27.0, 'acc': 2, 'v_0': 14.0, 'x_start': 459.0}, {'t_start': 27.0, 'acc': 0, 'v_0': 20.0, 'x_start': 510.0, 't_end': 10000000.0}]
新しいETA:  [{'waypoint_idx': 0, 'x': 0.0, 'eta': 0.0, 'car_idx': 0, 'type': 'waypoint'}, {'waypoint_idx': 1, 'x': 150.0, 'eta': 7.5, 'car_idx': 0, 'type': 'waypoint'}, {'waypoint_idx': 2, 'x': 300.0, 'eta': 15.0, 'car_idx': 0, 'type': 'waypoint'}, {'waypoint_idx': 3, 'x': 450.0, 'eta': 23.35713316333455, 'car_idx': 0, 'type': 'waypoint'}, {'waypoint_idx': 4, 'x': 600.0, 'eta': 31.4999775001125, 'car_idx': 0, 'type': 'waypoint'}, {'waypoint_idx': 5, 'x': 750.0, 'eta': 38.9999400003, 'car_i

In [4]:
df = simulation.reservation_table.eta_table
df = df[df["car_idx"] == 0]
df

,waypoint_idx,x,eta,car_idx,type
car_idx,,,,,
0,0,0.0,0.000000,0,waypoint
0,1,150.0,7.500000,0,waypoint
0,2,300.0,15.000000,0,waypoint
0,3,450.0,23.357133,0,waypoint
0,4,600.0,31.499978,0,waypoint
0,5,750.0,39.055728,0,waypoint
0,6,900.0,47.999990,0,waypoint
0,7,1050.0,55.499953,0,waypoint
0,8,1200.0,64.486833,0,waypoint


In [6]:
car_1 = simulation.CARS[1]
print(car_1.my_etas)

[{'waypoint_idx': 0, 'x': 0.0, 'eta': 3.0, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 1, 'x': 150.0, 'eta': 10.5, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 2, 'x': 300.0, 'eta': 18.0, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 3, 'x': 450.0, 'eta': 25.5, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 4, 'x': 600.0, 'eta': 33.0, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 5, 'x': 750.0, 'eta': 40.852940604632266, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 6, 'x': 900.0, 'eta': 50.5061612099793, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 7, 'x': 1050.0, 'eta': 58.371337569739474, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 8, 'x': 1200.0, 'eta': 66.52351438852297, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 9, 'x': 1350.0, 'eta': 74.05548241208989, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 10, 'x': 1500.0, 'eta': 82.44795778565559, 'car_idx': 1, 'type': 'waypoint'}, {'waypoint_idx': 11, 'x': 1650.0, 'eta